# Meta Feature Engineering

Within this Notebook we directly go on with the result of last time. So we are taking the features of last time and try to mutate them the way, that the models can gain the most insights of them. 

In the end we again output a ```static_features.pk``` and ```dynamic_features.pk``` file. The reason to not combine them in advance, is because different models expect different preprocessing. The joining for example of the two features, like needed for XGboost is then done in the respective notebook.
A second reason for this is the saving of Google Drive space. This is fundamental important cause... mine is also completely full... :D   

# 1. Setting up Colab

Here we set up colab. U knooow it :D 

In case u don't use colab, please specify the variable ```use_colab``` to false. In case u don't use it its unavoidable to make sure that noone changed something online while u change it offline. The results of this would be worse than a division by zero.  

In [ ]:
use_colab: bool = True

In case u accidentally run the following code twice u will get following weird and confusing error:

```
OSError: [Errno 107] Transport endpoint is not connected
```

In this case just restart the runtime above. 


In [ ]:
if use_colab:
  import os
  from google.colab import drive 
  drive.mount("/content/gdrive", force_remount=True)
  # Change directory for nicer imports
  %cd "/content/gdrive/My Drive/Data_Mining_Cup/05 Code/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Data_Mining_Cup/05 Code


# 2. Nodes and Ideas

## Ideas for new Features
Please specify assignment in bold and brackets behind. If nobody is assigned put the "To Be Assigned (TBA)" Flage.

## Notes

Currently no notes :( 

# 3. Setting up Notebook

Here we setup some parameters for the notebook to work in the expected ways.

If we want to **use the final test phase** for the later submission, we need to specify the following parameter **```make_final_submission``` to true.** Otherwise the selfmade test set will be used.

In [ ]:
make_final_submission: bool = True

In case we don't want to make a final submission, we now have to specify the test_set. In case you change it, please leave a comment on why and what the new one is. If the end is defined as None we use all the data from start_period (inclusive) till the end of dataset. 

In case make_final_submission is True we do not use those parameters.

In [ ]:
# This will set the start of test period to 2 weeks before end of data

test_period_start: str="2018-06-02"
# Use till the end
test_period_end: str="2018-06-15"

In [ ]:
# These datapoints are for the final submision
final_submission_start: str="2018-06-30"
final_submission_end: str="2018-07-13"

#### Imports

Here you can add imports u need. Please use alphabetical order to not import new stuff twice.

In [ ]:

from datetime import datetime
from google.colab import drive
from keras.preprocessing.sequence import TimeseriesGenerator
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from typing import List, Callable

%matplotlib inline

### Data Imports

Now we import the source data. In case we use our test set, we delete the test set values to simulate later submission.

In [ ]:
# OPEN PANDAS DATAFRAMES FROM PREPROCESSING 1
try:
  if make_final_submission:
    # In tables 'infos' and 'items' itemID is used as index
    
    dynamic_features = pd.read_pickle('./data/dynamic_features_2906_final.pk')
    static_features = pd.read_pickle('./data/static_features_2906_final.pk')
  else:
    dynamic_features = pd.read_pickle('./data/dynamic_feature_2906.pk')
    static_features = pd.read_pickle('./data/static_feature_2906.pk')
except FileNotFoundError as e:
  print(f"One File could not be found. Please rerun the first Notebook, {e}")

# 6. Defining Meta Feature Functions

In this area new meta features can be defined. A meta feature function takes the data from the first notebook and applies mutation on them

__Why do we need functions?__

See Notebook 1.

__What if my function needs additional parameters?__

Then create a function which returns a the final function. The following function returns a function which changes the sales price to the default value + 10.

```
def test_function(default_value=1):

  def final_function(data: pd.DataFrame):
    data['salesPrice'] = default_value + 10

    return data
  return final_function

```
Later this function can be triggered using following:

```
test_function(default_value=10)(data)
```
This structure is important for later and enables adding the same feature with only different parameter.


# Method Definitions

Here the methods for mutating the dataframe are defined

#### method of one hot encoding

In [ ]:
def one_hot_encoding(field: str, delete_old: bool = True)->pd.DataFrame:
  """
  replaces columns with one hot encoding of it.
  If the parameter delete_old is True (this is defaultly the case), 
  the originally column is deleted. 
  """
  def one_hot_fct(df: pd.DataFrame):
    dummy = pd.get_dummies(df[field], prefix=f"{field}_", drop_first=True)
    
    df = pd.concat([df, dummy], axis=1)
    if delete_old: 
      df = df.drop(field, axis=1)

    return df

  return one_hot_fct

#### One Differencing (not implemented yet)

In [ ]:
def one_difference(dataframe: pd.DataFrame, column):
  pass

In [ ]:
# prepare_data_frame_per_item(orders, 3).sort_values(by="date").tail()

#### Make the crazy clustering
 
 TODO ADD To Notebook 1 

In [ ]:
def k_means_clustering(data, **kwargs):
  """
  Does a clustering on the timeseries data.

  Returns a clustering mapping ItemID -> cluster
  """
  DIMENSIONS_FOR_CLUSTERING = kwargs.get('dimensions', 10)
  CLUSTERS_FOR_CLUSTERING = kwargs.get('clusters', 4)
  print(f"...create clustering with {CLUSTERS_FOR_CLUSTERING} clusters based on reduced {DIMENSIONS_FOR_CLUSTERING} dimensions")

  columns = data['date'].unique()
  idx = data['itemID'].unique()
  df = pd.DataFrame(index=idx,columns=columns)
  
  for i in columns:
    df[i] = data[data['date']==i][['order','itemID']].groupby('itemID').sum()

  # Apply PCR

  pca_show = PCA(n_components=3)
  low_dim_show = pd.DataFrame(pca_show.fit_transform(df.values))

  pca_cluster = PCA(n_components=DIMENSIONS_FOR_CLUSTERING)
  low_dim_cluster=pd.DataFrame(pca_cluster.fit_transform(df.values))

   # Clustering
  kmeans = KMeans(n_clusters=4, random_state=0)
  kmeans.fit(low_dim_cluster)
  clusters = kmeans.predict(low_dim_cluster)
  low_dim_show['cluster'] = clusters

  fig = plt.figure()
  ax = Axes3D(fig)
  ax.scatter(low_dim_show.loc[:,0], low_dim_show.loc[:,1], low_dim_show.loc[:,2], c=low_dim_show['cluster'])
 
  low_dim_show.index=low_dim_show.index+1
  df['cluster_group'] = low_dim_show['cluster']
  return df['cluster_group']

#### Convert Bool to int

In [ ]:
def convert_to_int(fields: List[str]):
  def convert_fct(data):
      
      data[fields] = data[fields].astype(int)
      return data

  return convert_fct

#### Sliding window

In [ ]:
def add_slinding_window(lag_size: int=7, fields: List[str] = None, delete_nan_in_window=True, suicidal=False):
  """
  Applies shifting on the data.
  New columns will have the suffix _t-1 for the first lag, etc
  """
  fields = fields or dataframe.columns
  def sliding_window(dataframe: pd.DataFrame):
    if lag_size > 10 and not suicidal:
      print("In Google Colab, this might be crazy. To not make u start the runtime over and over again, and finally fall into a crazy depression, I won't do it. \n In case that u are really really really sure, then u can pass the 'sucicidal' argument with a true value....a")
      return 

    assert isinstance(dataframe, pd.DataFrame), "incoming data needs to be of type pandas dataFrame"

    for lag in range(1, lag_size+1):
      lag_features =  dataframe.groupby(level=0)[fields].shift(lag).add_suffix(f"_t-{lag}")
      dataframe = pd.concat([dataframe,lag_features], axis=1)
      
      if delete_nan_in_window:
        dataframe = dataframe.dropna(subset=list(lag_features.columns))

    return dataframe

  return sliding_window

# Method Applying

In here the methods will be applied according to the dict on the top

# Which stuff should be applied

The following dict describes which methods should be applied to the data

In [ ]:
applied_on_dynamic_features: List[Callable] = [
  convert_to_int(fields=['promoted', 'promotion_lag1','promotion_lag2','two_days_after_promo','de_holidays']),
  one_hot_encoding('salesPrice_bucket'),
  one_hot_encoding('day_of_week'),
  add_slinding_window(lag_size=7,
                      fields=[
                              "order"
                      ],
                      delete_nan_in_window=False, 
                      suicidal=True)                          
]
applied_on_static_features: List[Callable] = [
  convert_to_int(fields=['has_rating','promotion_in_data']),
  one_hot_encoding('category1'), 
  one_hot_encoding('category2'), 
  one_hot_encoding('category3'), 
  one_hot_encoding('manufacturer'), 
  one_hot_encoding('brand'),
  one_hot_encoding('recommendedRetailPrice_bucket'),
  # one_hot_encoding('price_sumSales_cluster'),
  one_hot_encoding('bought_together_cluster')
]


In [ ]:
dynamic_features

order  salesPrice  ...  last_promoted_sales  salesPrice_bucket
itemID date                           ...                                        
1      2018-01-01    0.0        8.84  ...                164.0                  1
       2018-01-02    0.0        8.84  ...                164.0                  1
       2018-01-03    0.0        8.84  ...                164.0                  1
       2018-01-04    0.0        8.84  ...                164.0                  1
       2018-01-05    0.0        8.84  ...                164.0                  1
...                  ...         ...  ...                  ...                ...
10463  2018-07-09    NaN      282.16  ...                 36.0                  9
       2018-07-10    NaN      282.16  ...                 36.0                  9
       2018-07-11    NaN      282.16  ...                 36.0                  9
       2018-07-12    NaN      282.16  ...                 36.0                  9
       2018-07-13    NaN      282.16  ...                 36.0                  9

[2029822 rows x 22 columns]

In [ ]:
def apply_feature_list(data: pd.DataFrame, callable_list: List[Callable])->pd.DataFrame:
  for fct in callable_list:
    result = fct(data)
    assert result is not None, "One feature function does not return anything..."
    data = result
  
  return data

dynamic_features = apply_feature_list(dynamic_features, applied_on_dynamic_features)
static_features = apply_feature_list(static_features, applied_on_static_features)

# Save dataset
Here the both datasets are saved.

The following variable is used to save the data

In [ ]:
suffix: str= "" #"update_one_hot"

In [ ]:
save_folder = "./data"

Following both datasets are saved. Please don't save other data from here. We are only using the two files as described above.

In [ ]:
def save_data(df: pd.DataFrame, filename: str):
  """
  Saves data to the pickle file.
  Sample call:
  save_data(items, "items.pk")
  """
  filename = f"{save_folder}/{filename}{suffix}"
  df.to_pickle(filename)


save_data(static_features, 'static_features_final_nb2_2906_one_hot_ws7.pk' if make_final_submission else 'static_features_nb2_2906_one_hot_ws7.pk')
save_data(dynamic_features, 'dynamic_features_final_nb2_2906_one_hot_ws7.pk' if make_final_submission else 'dynamic_features_nb2_2906_one_hot_ws7.pk')

# Final Data

In [ ]:
static_features

,customerRating,recommendedRetailPrice,avg_salesPrice_nonunique,avg_salesPrice,avg_sold_promoted,has_rating,num_promotions,promotion_in_data,avg_sold_not_promoted,SUM(orders.order),SUM(orders.salesPrice),STD(orders.order),STD(orders.salesPrice),MAX(orders.order),MAX(orders.salesPrice),SKEW(orders.order),SKEW(orders.salesPrice),MIN(orders.order),MIN(orders.salesPrice),MEAN(orders.order),MEAN(orders.salesPrice),COUNT(orders),NUM_UNIQUE(orders.DAY(time)),NUM_UNIQUE(orders.DAY(date)),NUM_UNIQUE(orders.MONTH(time)),NUM_UNIQUE(orders.MONTH(date)),NUM_UNIQUE(orders.WEEKDAY(time)),NUM_UNIQUE(orders.WEEKDAY(date)),MODE(orders.DAY(time)),MODE(orders.DAY(date)),MODE(orders.MONTH(time)),MODE(orders.MONTH(date)),MODE(orders.WEEKDAY(time)),MODE(orders.WEEKDAY(date)),category1__2,category1__3,category1__4,category1__5,category1__6,category1__7,...,brand__274,recommendedRetailPrice_bucket__1,recommendedRetailPrice_bucket__2,recommendedRetailPrice_bucket__3,recommendedRetailPrice_bucket__4,recommendedRetailPrice_bucket__5,recommendedRetailPrice_bucket__6,recommendedRetailPrice_bucket__7,recommendedRetailPrice_bucket__8,recommendedRetailPrice_bucket__9,bought_together_cluster__0,bought_together_cluster__2,bought_together_cluster__5,bought_together_cluster__6,bought_together_cluster__8,bought_together_cluster__12,bought_together_cluster__13,bought_together_cluster__16,bought_together_cluster__21,bought_together_cluster__24,bought_together_cluster__26,bought_together_cluster__27,bought_together_cluster__28,bought_together_cluster__31,bought_together_cluster__32,bought_together_cluster__33,bought_together_cluster__34,bought_together_cluster__37,bought_together_cluster__38,bought_together_cluster__39,bought_together_cluster__40,bought_together_cluster__43,bought_together_cluster__46,bought_together_cluster__47,bought_together_cluster__49,bought_together_cluster__50,bought_together_cluster__51,bought_together_cluster__52,bought_together_cluster__53,bought_together_cluster__55
itemID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.38,8.84,3.111661,3.116335,164.500000,1,4.0,1,0.198864,693.0,1798.54,0.482123,0.023014,4.0,3.43,2.613255,13.808001,1.0,3.11,1.198962,3.111661,578.0,20.0,20.0,6.0,6.0,7.0,7.0,29.0,29.0,1.0,1.0,0.0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3.00,16.92,9.150000,9.150000,207.929325,1,0.0,0,0.027778,5.0,45.75,0.000000,0.000000,1.0,9.15,0.000000,0.000000,1.0,9.15,1.000000,9.150000,5.0,4.0,4.0,2.0,2.0,4.0,4.0,29.0,29.0,5.0,5.0,1.0,1.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5.00,15.89,12.733253,12.791506,105.666667,1,3.0,1,0.079096,331.0,3679.91,0.399201,1.930884,4.0,14.04,3.138375,-0.801297,1.0,9.89,1.145329,12.733253,289.0,15.0,15.0,5.0,5.0,7.0,7.0,28.0,28.0,6.0,6.0,3.0,3.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4.44,40.17,13.798895,13.771020,92.000000,1,2.0,1,0.078652,198.0,2497.60,0.310942,0.488733,3.0,14.10,3.353433,-1.009222,1.0,13.01,1.093923,13.798895,181.0,10.0,10.0,5.0,5.0,5.0,5.0,19.0,19.0,6.0,6.0,1.0,1.0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2.33,17.04,7.735556,7.726392,81.500000,1,2.0,1,0.095506,180.0,1253.16,0.334367,0.163881,3.0,7.84,2.990726,-0.933601,1.0,7.48,1.111111,7.735556,162.0,9.0,9.0,5.0,5.0,7.0,7.0,17.0,17.0,4.0,4.0,1.0,1.0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10459,0.00,56.57,14.710000,14.710000,61.214153,0,0.0,0,0.005556,1.0,14.71,NaN,NaN,1.0,14.71,NaN,NaN,1

In [ ]:
dynamic_features

order  salesPrice  promoted  ...  order_t-5  order_t-6  order_t-7
itemID date                                     ...                                 
1      2018-01-01    0.0        8.84         0  ...        NaN        NaN        NaN
       2018-01-02    0.0        8.84         0  ...        NaN        NaN        NaN
       2018-01-03    0.0        8.84         0  ...        NaN        NaN        NaN
       2018-01-04    0.0        8.84         0  ...        NaN        NaN        NaN
       2018-01-05    0.0        8.84         0  ...        NaN        NaN        NaN
...                  ...         ...       ...  ...        ...        ...        ...
10463  2018-07-09    NaN      282.16         0  ...        NaN        NaN        NaN
       2018-07-10    NaN      282.16         0  ...        NaN        NaN        NaN
       2018-07-11    NaN      282.16         0  ...        NaN        NaN        NaN
       2018-07-12    NaN      282.16         0  ...        NaN        NaN        NaN
       2018-07-13    NaN      282.16         0  ...        NaN        NaN        NaN

[2029822 rows x 42 columns]